In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import warnings
warnings.filterwarnings('ignore')

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# The dataset shareable link

#new stylo data

link = 'https://drive.google.com/open?id=1qg3fx5JyibrfBBycl3y8hCQJWBiXkP2N'


fluff, id = link.split('=')
print(id)
filename = 'stylo-data.csv'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile(filename)

#raw_data_link = 'https://drive.google.com/open?id=1DZkGVcBNQ1WUk11dVRzJcbGNeAb8rOaJ'
#stopwords
raw_data_link = 'https://drive.google.com/open?id=1QKJm85j3xeOz4iIgd5D82i7tc3vg0zMW'
fluff, id = raw_data_link.split('=')
print(id)
raw_filename = 'raw_data.csv'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile(raw_filename)


1qg3fx5JyibrfBBycl3y8hCQJWBiXkP2N
1QKJm85j3xeOz4iIgd5D82i7tc3vg0zMW


In [0]:
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt

import nltk
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

!pip install -U keras
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.layers import InputLayer, Embedding, Conv1D, MaxPool1D, Dense, Flatten, Dropout, GRU, LSTM, Input
from tensorflow.python.keras import Sequential, Model
from tensorflow.python.keras import Sequential
from keras.initializers import Constant

from pprint import pprint

from imblearn.over_sampling import SMOTE

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


*PS: Para baixar os arquivos do Google Drive, é preciso clicar no link que irá aparecer no console, aceitar as permissões do Drive e em seguida colar o código de verificação no local indicado (console)*

*Sem o arquivo não é possível executar os outros trechos do notebook*

In [0]:
# read dataset
df = pd.read_csv(filename)
raw_df = pd.read_csv(raw_filename)

# Remove single class entries (We need at least 3 samples of an author)#
df = df.groupby('Author').filter(lambda x: len(x) > 3)
raw_df = raw_df.groupby('Author').filter(lambda x: len(x) > 3)

In [5]:
# Visualize data


df.shape
df.head()
df.shape
raw_df.head()

,Text,Author
1,respostas ecologia evolutiva retirar amostra p...,pdje
2,1 - competição membros espécies diferentes usa...,fadj
3,embora temática questão ambiental crescente an...,sdjme
5,vem cada dia buscando entender relações estabe...,sdfi
6,"pode-se dizer que, conservar dado habitat dada...",bmm


In [0]:
stopwords = nltk.corpus.stopwords.words('portuguese')
raw_df['Text'] = raw_df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stopwords)]))

In [0]:
raw_df = raw_df.replace(to_replace ='\r', value = ' ', regex = True) 
raw_df = raw_df.replace(to_replace ='\n ', value = '\n', regex = True) 
raw_df = raw_df.replace(to_replace ='\n{2,*}', value = ' ', regex = True)

In [33]:
raw_df['Text']

1      respostas ecologia evolutiva retirar amostra p...
2      1 - competição membros espécies diferentes usa...
3      embora temática questão ambiental crescente an...
5      vem cada dia buscando entender relações estabe...
6      pode-se dizer que, conservar dado habitat dada...
                             ...                        
92     referencial teórico: anuras (bufonídeos) sapos...
93     congregação santa – – segunda conferência orga...
94     congregação santa biológicas educação ambienta...
95     disciplina: orientação supervisão estágio ii p...
102    biologia tendências perspectivas atuais aborda...
Name: Text, Length: 69, dtype: object

In [0]:
#Separate data from labels

y = df.pop('Author')
y_raw = raw_df.pop('Author')
le = LabelEncoder()
le.fit(y)
encoded_Y = le.transform(y)

le.fit(y_raw)
encoded_raw_Y = le.transform(y_raw)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer

tfidf_transformer = TfidfTransformer()
X_idf = tfidf_transformer.fit_transform(raw_df)

In [0]:

gx = GaussianMixture()

#gx.fit(df)
n_components = np.arange(1, 30)
models = [GaussianMixture(n, covariance_type='full', random_state=0).fit(df) for n in n_components]
plt.plot(n_components, [m.bic(df) for m in models], label='BIC')
plt.plot(n_components, [m.aic(df) for m in models], label='AIC')
plt.legend(loc='best')
plt.xlabel('n_components');

In [0]:
gmm = GaussianMixture(n_components=5)
gmm.fit(df)
labels = gmm.predict(df.values)
plt.scatter(df.values[:, 0], df.values[:, 6], c=labels, cmap='viridis');

In [0]:
labels

In [0]:
scaler = StandardScaler()
# Fit on training set only.
x = scaler.fit_transform(df)

pca = PCA(0.95)
X = pca.fit_transform(x)

In [0]:
#W2Vec tryout
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# count_vect = CountVectorizer()
# word_count_vector = count_vect.fit_transform(raw_df['Text'])
 
# settings that you use for count vectorizer will go here
#tfidf_vectorizer=TfidfVectorizer(use_idf=True)
 
# just send in all your docs here
#tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(raw_df['Text'])

# tfidf_transformer = TfidfTransformer()
# X_raw = tfidf_transformer.fit_transform(X_train_counts)
#tfidf_vectorizer_vectors[0].to

# for train_index, test_index in kfold.split(X_raw, encoded_raw_Y):
#   X_train, y_train = X_raw[train_index], encoded_raw_Y[train_index]
#   X_test, y_test = X_raw[test_index], encoded_raw_Y[test_index]
#   print(X_raw[train_index])



X_train, X_test, y_train, y_test = train_test_split(raw_df['Text'], encoded_raw_Y, random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [0]:
stopwords = nltk.corpus.stopwords.words('portuguese')
#models = [AdaBoostClassifier(), BaggingClassifier(),ExtraTreesClassifier(), GradientBoostingClassifier(),
           #RandomForestClassifier(), SGDClassifier(), LinearSVC(), SVC(), BernoulliNB(), MultinomialNB(), MLPClassifier() ]

models = [ExtraTreesClassifier(), RandomForestClassifier(), SGDClassifier(), MultinomialNB(), MLPClassifier()]
names = ['extratrees', 'randomf', 'sgdclassfier','multinomialNB', 'mlp']

results = []
accuracies = {}
for idx, model in enumerate(models):
  for ngram in range(1,5):
    clf = make_pipeline(TfidfVectorizer(use_idf=True, stop_words = stopwords, ngram_range=(1, ngram)), model)
    accs = []
    for i in range(0, 10):
      scores = cross_validate(clf, raw_df['Text'], encoded_raw_Y, scoring=['accuracy'], cv=3, return_train_score=True)
      #print(np.mean(scores['test_accuracy']))
      #print(component, names[idx])
      #print(scores['test_accuracy'])
      mean = np.mean(scores['test_accuracy'])
    accs.append(mean)
    results.append((np.mean(accs), ngram, names[idx]))

print(results)
results

In [0]:
# Data normalization
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer, QuantileTransformer, Normalizer

scaled_features = PowerTransformer().fit_transform(df.values)
df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)


In [0]:
X, X_test, Y, y_test = train_test_split(df.values, encoded_Y, random_state=7, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, random_state=7, test_size=0.5)

model = RandomForestClassifier()
print(X.shape)
model.fit(X, Y)

print(X_test.shape)
y_pred = model.predict(X_test)

print(y_pred)
print(y_test)
score = accuracy_score(y_test, y_pred)
print("Test score: %.2f%%" % (score * 100))

y_pred = model.predict(X_val)
print(y_pred)
print(y_val)
score = accuracy_score(y_val, y_pred)
print("Validation score: %.2f%%" % (score * 100))

In [0]:
df.shape

In [0]:
#stopwords = nltk.corpus.stopwords.words('portuguese')
models = [ExtraTreesClassifier(), 
           RandomForestClassifier(), SVC(kernel='poly'), SVC(), MLPClassifier() ]

names = ['extratrees','randomf', 'svcpoly', 'svcrbf', 'mlp']

#results = []
results2 = []
accuracies = {}
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=7)
for idx, model in enumerate(models):
  for component in range(5, 48):
    clf = make_pipeline(PCA(n_components=component), model)
    accs = []
    for i in range(0, 10):
      scores = []
      for train_index, test_index in kfold.split(df, encoded_Y):
        X_train, y_train = df.iloc[train_index], encoded_Y[train_index]
        X_test, y_test = df.iloc[test_index], encoded_Y[test_index]
        #X_train_oversampled, y_train_oversampled = SMOTE(k_neighbors=2).fit_sample(X_train, y_train)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        scores.append(accuracy_score(y_test, y_pred))
        #print(np.mean(scores['test_accuracy']))
        #print(component, names[idx])
        #print(scores['test_accuracy'])
      mean = np.mean(scores)
      accs.append(mean)
    results2.append((np.mean(accs), component, names[idx]))
print(results2)

In [0]:
np.mean(accs)

In [0]:
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)
Y = encoded_Y
accuracies = []
variances = []
X = df

models = [ExtraTreesClassifier(), RandomForestClassifier(), MLPClassifier(), SVC(kernel='poly'), MLP()]
for i in range(0,100):
  scores = []
  for train_index, test_index in kfold.split(X, Y):
      #convert integers to dummy variables (i.e. one hot encoded)
      #dummy_y = np_utils.to_categorical(Y)
      X_train, y_train = X.iloc[train_index], Y[train_index]
      X_test, y_test = X.iloc[test_index], Y[test_index]
      X_train_oversampled, y_train_oversampled = SMOTE(k_neighbors=1).fit_sample(X_train, y_train)
      #rfclf = LinearSVC(multi_class='crammer_singer')
      
      #rfclf = KNeighborsClassifier(n_neighbors=5)
      #rclf = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=5), n_estimators=100, random_state=0)
      #rfclf = ExtraTreesClassifier(max_depth=None, n_estimators=500)
      rfclf = LogisticRegression(max_iter=500,solver ='lbfgs')
      #rfclf = RandomForestClassifier(n_estimators=500)
      rfclf.fit(X_train_oversampled, y_train_oversampled)
      y_pred = rfclf.predict(X_test)
      #print(confusion_matrix(y_test,y_pred))
      #print(classification_report(y_test,y_pred))
      #print(accuracy_score(y_test, y_pred))
      scores.append(accuracy_score(y_test, y_pred))

  accuracies.append(np.mean(scores))
print(np.mean(accuracies))

In [0]:
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=200, random_state=0)
scores = cross_val_score(clf, X, Y, cv=4)
scores

In [0]:
# Oversampling 
from imblearn.over_sampling import SMOTE


smt = SMOTE(k_neighbors=1)
X_resampled, y_resampled = smt.fit_resample(X_train, y_train)

X_resampled, y_resampled

In [0]:
# Data split

X_train, X_test, y_train, y_test = train_test_split(df, encoded_Y, test_size=0.3, random_state=0)
#X_test, X_application, y_test, y_application = train_test_split(X_test, y_test, test_size=0.5, random_state=0)

y_train

In [0]:
#Random Forest baseline + accuracy versus test
rfclf = RandomForestClassifier(n_estimators=200, random_state=0)
rfclf.fit(X_resampled, y_resampled)
y_pred = rfclf.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

In [0]:
# Random Forest hyperparam discovery

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 800, num = 10)]
# Number of features to consider at every split
#max_features = ['auto', 'sqrt']
loss = ['deviance', 'exponential']
learning_rate = [0.1, 0.2, 0.3, 0.4,0.5]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [0, 1, 2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [0, 1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               #'loss': loss,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
               #'learning_rate': learning_rate}
               #'bootstrap': bootstrap}

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)
Y = encoded_Y
scores = []
model = []
best_params = []
X = df
for train_index, test_index in kfold.split(X, Y):
    #convert integers to dummy variables (i.e. one hot encoded)
    #dummy_y = np_utils.to_categorical(Y)
    X_train, y_train = X.iloc[train_index], Y[train_index]
    X_test, y_test = X.iloc[test_index], Y[test_index]
    X_train_oversampled, y_train_oversampled = SMOTE(k_neighbors=1).fit_sample(X_train, y_train)
    hrfclf = ExtraTreesClassifier()
    rf_random = RandomizedSearchCV(estimator = hrfclf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    rf_random.fit(X_train_oversampled, y_train_oversampled)
    model.append(rf_random.best_estimator_)
    best_params.append(rf_random.best_params_)
    y_pred = rf_random.best_estimator_.predict(X_test)
    #print(confusion_matrix(y_test,y_pred))
    #print(classification_report(y_test,y_pred))
    print(accuracy_score(y_test, y_pred))
    scores.append(accuracy_score(y_test, y_pred))


# Fit the random search model
#rf_random.fit(X_resampled, y_resampled)

In [0]:
best_params

In [0]:
y_pred = rf_random.best_estimator_.predict(X_test)
print(accuracy_score(y_test, y_pred))

In [0]:
# RFC w/ tunned HP
rfclf = RandomForestClassifier(bootstrap='True', max_depth=20, max_features='sqrt', n_estimators=900, random_state=0)
rfclf.fit(X_resampled, y_resampled)
pred= rfclf.predict(X_test)
accuracy_score(pred, y_test)

In [0]:
glove_100_link = 'https://drive.google.com/open?id=1hW4DhKFWJJPNIEI3GelC1WLzEqU-lrub'

glove_50_link = 'https://drive.google.com/open?id=10MW2F53DYYxizSXLbvfsphs77Fu1U9Gz'

skip_text_link = 'https://drive.google.com/open?id=1WXpYIhDI8Hhne3cOZUQshScjwsWO2wRR'

cbow_link = 'https://drive.google.com/open?id=1o6bO6QDAOFd_bASWuZHfbBlk8OBNA0uQ'

def load_file(link, filename):
  # Download gloVe word embedding for portuguese-BR (100-dimensions)
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  fluff, id = link.split('=')
  downloaded = drive.CreateFile({'id':id})
  downloaded.GetContentFile(filename)

def build_embedding_dict(filename):
  embd_dict = {}
  with open(filename, 'r', encoding='utf-8') as f:
    for line in f:
      word, coefs = line.split(maxsplit=1)
      coefs = np.fromstring(coefs, 'f', sep=' ')
      embd_dict[word] = coefs
  return embd_dict
  

In [0]:
# Download and load embeddins sinto a dictionaries (eg: {'oi' : n1, n2, ..., n100})

load_file(glove_100_link, 'glove_100.txt')
load_file(glove_50_link, 'glove_50.txt')
load_file(cbow_link, 'cbow_50.txt')
load_file(skip_text_link, 'skipgram_50.txt')

word_embedding_dict_50 = build_embedding_dict('glove_50.txt')
word_embedding_dict_100 = build_embedding_dict('glove_100.txt')
word_embedding_cbow_50 = build_embedding_dict('cbow_50.txt')
word_embedding_skipgram_50 = build_embedding_dict('skipgram_50.txt')

In [12]:
corpus = raw_df['Text']

# Create a Keras tokenizer and convert words to integer values
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(corpus)

# Convert strings to their corresponding integer value created by the Tokenizer
embedded_sentences = tokenizer.texts_to_sequences(corpus)
print(embedded_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[[758, 203, 759, 2664, 2665, 1117, 510, 2666, 902, 2667, 1402, 2668, 222, 190, 13, 141, 1118, 366, 2669, 223, 224, 110, 2670, 574, 2671, 1830, 279, 222, 1403, 168, 511, 512, 1119, 575, 5, 38, 1831, 576, 2672, 1832, 903, 2673, 512, 2674, 169, 204, 575, 38, 1831, 190, 8, 576, 2675, 513, 169, 2676, 1118, 124, 2677, 903, 760, 1120, 168, 225, 1833, 1120, 575, 5, 179, 13, 2678, 1834, 1835, 367, 170, 406, 2679, 1121, 1404, 2680, 903, 2681, 904, 169, 204, 223, 1122, 512, 761, 38, 2682, 236, 171, 205, 2683, 514, 1405, 1836, 11, 45, 2684, 654, 1830, 574, 905, 762, 2685, 222, 237, 2686, 1123, 222, 11, 1837, 1406, 238, 280, 2687, 906, 10, 2688, 124, 1122, 238, 170, 512, 2689, 1838, 1405, 1404, 655, 111, 512, 1122, 1838, 1405, 2690, 13, 45, 1117, 907, 1834, 1835, 2691, 577, 903, 2692, 2693, 574, 223, 908, 1121, 1122, 2694, 13, 1837, 909, 124, 905, 237, 205, 2695, 656, 2696, 1836, 13, 46, 1

In [0]:
corpus

In [13]:
max_words_text = max(corpus, key=lambda sentence: len(nltk.word_tokenize(sentence, language='portuguese')))
print(max_words_text[:200])

biologia tendências perspectivas atuais abordagem sala aula texto disponibilizado sala aula virtual, histórico ciências, deve ter percebido modificações ocorridas estreita relação transformações política, econômica, social cultural mundo país. desse modo, naturais vem sendo modificado sistematicamente escolas, adaptando-se diferentes modos organização social variadas concepções pedagógicas. diferentes modos organização social vivemos hoje, diretamente ligados mudanças transformações aceleradas mundo. podemos perceber mudanças ocorrendo apenas redor nós, ocorrem interior, forma conceber representar mundo (lévy, 1993). dessa forma, vamos percebendo modificações apresentam via mão dupla, transformamos transformados através situações vão ocorrendo vida. últimos anos, deve ter percebido ênfase sendo dada exercício cidadania, principalmente tocante escola. sido norteado lei diretrizes base educação nacional, nº 9.394/96, art. 2 0 diz: ”a educação, dever família estado, inspirada princípios l

In [14]:
max_sentence_len = len(nltk.word_tokenize(max_words_text, language='portuguese'))
print(max_sentence_len)

3913


In [15]:
# Make all the sentences have the same length by padding with 0s at the end
padded_sentences = pad_sequences(embedded_sentences, max_sentence_len, padding='post')

padded_sentences.shape

(69, 3913)

In [0]:
vocab_len = len(tokenizer.word_index) + 1

# Create an embedding matrix, in other words, use the weights from gloVe vector against all our corpus
# In the end we will have a matrix of all our vocabulary x N dimensions
def build_embd_matrix(size, embd_dic):
  # initialize with all zeros
  embedding_matrix = np.zeros((vocab_len, size))
  for word, index in tokenizer.word_index.items():
    embedding_vector = embd_dic.get(word)
    # if the word is in gloVe, we will use its weigths 
    if embedding_vector is not None:
      embedding_matrix[index] = embedding_vector[:size]
  return embedding_matrix


embedding_matrix_50 = build_embd_matrix(50, word_embedding_dict_50)

embedding_matrix_100 = build_embd_matrix(100, word_embedding_dict_100)

embedding_matrix_cbow_50 = build_embd_matrix(50, word_embedding_cbow_50)

embedding_matrix_skipgram_50 = build_embd_matrix(50, word_embedding_skipgram_50)

In [0]:
# TODO: Update this model to user TensorFlow 2.x
model = Sequential()
model.add(Embedding(vocab_len, 50,weights=[embedding_matrix], input_length=max_sentence_len))
model.add(Dropout(0.5))
model.add(Conv1D(64, 5, activation='relu'))

model.add(MaxPool1D(2, strides=2))
model.add(Flatten())
model.add(Dense(11, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc'])

In [0]:
model.metrics

In [0]:
X = padded_sentences
Y = np_utils.to_categorical(encoded_Y)
cnn_model.fit(X, Y, epochs=50, verbose=1)

In [0]:
def build_model(embd_matrix, size=50):
  model = Sequential()
  embedding_layer = Embedding(vocab_len, 50, input_length=max_sentence_len, 
                              embeddings_initializer=Constant(embedding_matrix_50), trainable=False)
  model.add(embedding_layer)
  model.add(Dropout(0.3))
  model.add(Flatten())
  number_of_classes = len(np_utils.to_categorical(encoded_raw_Y)[0])
  model.add(Dense(number_of_classes, activation='softmax'))

  model.compile(optimizer='rmsprop',
                    loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [0]:
model.layers

In [0]:
cv_scores = []
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)
models = []

X = padded_sentences

X, X_val, Y_train, y_val = train_test_split(padded_sentences, encoded_raw_Y, stratify=encoded_raw_Y, random_state=11, test_size=0.20)
model = build_model()
accs = []
for i in range(0,10):
  # stratified k-fold validation
  for train_index, test_index in kfold.split(X, Y_train):
    # Convert integers to vectors (e.g One hot encoding)
    Y = np_utils.to_categorical(Y_train)

    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    # train the model
    model.fit(X_train, y_train, epochs=50, verbose=0)

    # Evaluate the model
    scores = model.evaluate(X_test, y_test)
    #print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
    cv_scores.append(scores[1] * 100)
  accs.append(np.mean(cv_scores))

# Brief summary of the scores
print("%.2f%% (+/- %.2f%%)" % (np.mean(accs), np.std(accs)))

In [0]:
y_val.shape
y_val_dummy = np_utils.to_categorical(y_val)
X_val.shape

In [0]:
scores = model.evaluate(X_val, y_val_dummy)
print("Test score: %.2f%%" % (scores[1] * 100))

In [0]:
#scores = model.evaluate(X_val, y_val_dummy)

X, X_val, Y_train, y_val = train_test_split(padded_sentences, encoded_raw_Y, random_state=11, stratify=encoded_raw_Y, test_size=0.20)

In [60]:
def test_model(iter, embedding, name, epochs):
  test_scores = []
  val_scores = []
  for i in range(iter):
    Y = np_utils.to_categorical(encoded_raw_Y)

    X, X_test, Y, y_test = train_test_split(padded_sentences, Y, stratify=Y, random_state=11, test_size=0.2)
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, random_state=11, test_size=0.5)

    model = build_model(embedding, 100)

    model.fit(X, Y, epochs=epochs, verbose=0)
    scores = model.evaluate(X_test, y_test)

    print("Test score " + name + ": %.2f%%" % (scores[1] * 100))
    test_scores.append(scores[1] * 100)

    scores = model.evaluate(X_val, y_val)
    print("Validation score " + name + ": %.2f%%" % (scores[1] * 100))
    val_scores.append(scores[1] * 100)
  print("Mean test scores:" + ": %.2f%%" % (np.mean(test_scores)))
  print("Mean val scores:" + ": %.2f%%" % (np.mean(val_scores)))

ERROR! Session/line number was not unique in database. History logging moved to new session 65


In [62]:
# Run and print models output
test_model(10,embedding_matrix_50, 'Glove 50D', 150)
test_model(10,embedding_matrix_100,'Glove 100D', 150 )
test_model(10,embedding_matrix_cbow_50, 'CBOW 50D', 150)
test_model(10,embedding_matrix_skipgram_50, 'SkipGram 50D', 150)

1/1 [==============================] - 0s 2ms/step - loss: 6.0006 - accuracy: 0.0000e+00
Test score Glove 50D: 0.00%
1/1 [==============================] - 0s 1ms/step - loss: 2.3094 - accuracy: 0.1429
Validation score Glove 50D: 14.29%
1/1 [==============================] - 0s 2ms/step - loss: 7.1147 - accuracy: 0.0000e+00
Test score Glove 50D: 0.00%
1/1 [==============================] - 0s 1ms/step - loss: 2.5297 - accuracy: 0.2857
Validation score Glove 50D: 28.57%
1/1 [==============================] - 0s 2ms/step - loss: 6.6109 - accuracy: 0.0000e+00
Test score Glove 50D: 0.00%
1/1 [==============================] - 0s 1ms/step - loss: 2.5558 - accuracy: 0.1429
Validation score Glove 50D: 14.29%
1/1 [==============================] - 0s 2ms/step - loss: 6.4495 - accuracy: 0.0000e+00
Test score Glove 50D: 0.00%
1/1 [==============================] - 0s 2ms/step - loss: 2.5137 - accuracy: 0.1429
Validation score Glove 50D: 14.29%
1/1 [==============================] - 0s 2ms/step -

In [0]:
# Brief summary of the scores
print("%.2f%% (+/- %.2f%%)" % (np.mean(cv_scores), np.std(cv_scores)))

Atuais questionamentos

1.   Como o algoritmo irá se comportar diante de um vobaculário não visto anteriormente ?   
  1.1. Utilizar os novos dados obtidos (últimas 2 atividades para validação do modelo após solucionar o problema acima)
2.   Será que por meio dessa abordagem estamos aprendendendo o estilo de escrita do usuário ou o conteúdo escrito?
  2.2. Seria possível unir features estilométricas em conjunto aos word embeddings?    


In [0]:
# Save and upload model

model_json = model.to_json()
with open('model-lstm-2.json', 'w') as json_file:
  json_file.write(model_json)
model.save_weights('model-lstm-2.h5')

uploaded = drive.CreateFile({'title': 'model-lstm-2.h5'})
uploaded.SetContentFile('model-lstm-2.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

uploaded = drive.CreateFile({'title': 'model-lstm-2.json'})
uploaded.SetContentFile('model-lstm-2.json')
uploaded.Upload()


In [0]:
!ls -la

In [0]:
from google.colab import files
files.download('model-lstm.h5')